## Creating a DataFrame

First we need to start a SparkSession:

In [0]:
from pyspark.sql import SparkSession

Start Spark Session

In [0]:
# May take a little while on a local computer
spark = SparkSession.builder.appName("Basics").getOrCreate()

In [0]:
# Let's get file was saved in Databricks File System (DBFS).
# The same file was saved in github repository.
df = spark.read.json('/FileStore/tables/people.json')

##### Showing the data

In [0]:
# Note how data is missing!
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [0]:
df.columns

Out[12]: ['age', 'name']

In [0]:
df.describe()

Out[13]: DataFrame[summary: string, age: string, name: string]

Some data types make it easier to infer schema (like tabular formats such as csv which we will show later).

However you often have to set the schema yourself if you aren't dealing with a .read method that doesn't have inferSchema() built-in.

Spark has all the tools you need for this, it just requires a very specific structure:

In [0]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

Next we need to create the list of Structure fields 
* param name: string, name of the field. 
* param dataType: :class:`DataType` of the field. 
* param nullable: boolean, whether the field can be null (None) or not.

In [0]:
data_schema = [StructField("age", IntegerType(), True), StructField("name", StringType(), True)]

In [0]:
final_struc = StructType(fields = data_schema)

In [0]:
df = spark.read.json('/FileStore/tables/people.json', schema=final_struc)

In [0]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Grabbing the data

In [0]:
df['age']

Out[20]: Column<'age'>

In [0]:
type(df['age'])

Out[21]: pyspark.sql.column.Column

In [0]:
df.select('age')

Out[22]: DataFrame[age: int]

In [0]:
type(df.select('age'))

Out[25]: pyspark.sql.dataframe.DataFrame

In [0]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [0]:
# Returns list of Row objects
df.head(2)

Out[27]: [Row(age=None, name='Michael'), Row(age=30, name='Andy')]

Multiple columns

In [0]:
df.select(['age', 'name'])

Out[28]: DataFrame[age: int, name: string]

In [0]:
df.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Creating new columns

In [0]:
# Adding a new column with a simple copy
df.withColumn('newAge', df['age']).show()

+----+-------+------+
| age|   name|newAge|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [0]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
# Simple rename
df.withColumnRenamed('age', 'superNewAge').show()

+-----------+-------+
|superNewAge|   name|
+-----------+-------+
|       null|Michael|
|         30|   Andy|
|         19| Justin|
+-----------+-------+



More complicated operations to create a new columns.

In [0]:
df.withColumn('doubleAge', df['age'] * 2).show()

+----+-------+---------+
| age|   name|doubleAge|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [0]:
df.withColumn('add_one_age', df['age'] + 1).show()

+----+-------+-----------+
| age|   name|add_one_age|
+----+-------+-----------+
|null|Michael|       null|
|  30|   Andy|         31|
|  19| Justin|         20|
+----+-------+-----------+



In [0]:
df.withColumn('half_age', df['age'] / 2).show()

+----+-------+--------+
| age|   name|half_age|
+----+-------+--------+
|null|Michael|    null|
|  30|   Andy|    15.0|
|  19| Justin|     9.5|
+----+-------+--------+



In [0]:
df.withColumn('half_age', df['age'] / 2)

Out[36]: DataFrame[age: int, name: string, half_age: double]

### Using SQL

To use SQL queries directly with the dataframe, you will need to register it to a temporary view:

In [0]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

In [0]:
sql_results = spark.sql("SELECT * FROM people")

In [0]:
sql_results

Out[39]: DataFrame[age: int, name: string]

In [0]:
sql_results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
spark.sql("SELECT * FROM people WHERE age=30").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

